<a href="https://colab.research.google.com/github/BarryLiu-97/Pytorch-Tutorial/blob/master/06_Dataset_and_DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用minibatch进行训练

梯度下降可以使用的方法：
- 所有数据全部都用：利用CPU和GPU的并行计算能力，计算速度较高
- 随机梯度下降：只用一个样本，随机性较好，能克服鞍点的问题
- MiniBatch: 随机性较好的同时运算速度也较高

In [1]:
import torch
from torch.utils.data import Dataset      #Dataset是一个抽象类，抽象类是不能实例化的，只能被其他子类继承
from torch.utils.data import DataLoader     #Pytorch帮助我们加载数据的类
import numpy as np

In [ ]:
from google.colab import files
files.upload() 

In [3]:
class DiabetesDataset(Dataset):
  def __init__(self, filepath):
    diabetes = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
    self.len = diabetes.shape[0]     #数据集有多少个样本，就是有几行数据
    self.x_data = torch.from_numpy(diabetes[:,:-1])
    self.y_data = torch.from_numpy(diabetes[:,[-1]])

  def __getitem__(self, index):
    """
    使得实例化DiabetesDataset之后能够进行下标操作。dataset[index]
    """
    return self.x_data[index], self.y_data[index]   #  return x,y 表示返回的是(x, y)这样的元组

  def __len__(self):
    """
    返回数据条数
    """
    return self.len

dataset = DiabetesDataset('diabetes.csv')
train_loader = DataLoader(dataset=dataset, batch_size=32,
              shuffle=True, num_workers=2)   #num_workers表示读取数据的时候是否要多进程，几个进程进行读取

In [4]:
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.linear1 = torch.nn.Linear(8, 6)
    self.linear2 = torch.nn.Linear(6, 4)
    self.linear3 = torch.nn.Linear(4, 1)
    self.sigmoid = torch.nn.Sigmoid()  #在此处可以更改激活函数

  def forward(self, x):
    x = self.sigmoid(self.linear1(x))
    x = self.sigmoid(self.linear2(x))
    x = self.sigmoid(self.linear3(x))  #relu会使得小于0的值输出为0，会有风险，所以最后一层一般不用relu
    return x

model = Model()

criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
for epoch in range(100):
  for i, data in enumerate(train_loader, 0):
    #1. Prepare data
    inputs, labels = data   #都是张量
    #2. Forward
    y_pred = model(inputs)
    loss = criterion(y_pred, labels)
    print(epoch, i, loss.item())
    #3. Backward
    optimizer.zero_grad()
    #4. Update
    optimizer.step()